## Adaugarea unui set de date nou pentru clasificare

Am folosit un set de date refertor la operatiile de cezariana (0 daca nu este nevoie sa se faca cezariana, si 1 daca este nevoie).
Ca si feature-uri sunt: Age, delevery number, Delivery number, Delivery time, blood of pressure, heart problem

In [4]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, optim
import numpy as np
#pandas- librărie pentru lucrul cu fișierele
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import normalize

In [5]:
!pwd
df=pd.read_csv("caesarian.csv", error_bad_lines=False, engine='python')

/home/ioana/faculta/si/si/NNtema2/proiect-tema-2


In [7]:
df.values[0:6, :]

array([[26,  2,  0,  1,  0,  1],
       [26,  2,  1,  1,  0,  0],
       [28,  1,  0,  2,  0,  0],
       [22,  2,  0,  1,  0,  1],
       [26,  1,  1,  0,  0,  0],
       [27,  2,  0,  1,  0,  0]])

## Read data and normalize it

In [8]:
#Dataset - o clasă din PyTorch foarte utilă gestionării seturilor de date
class CaesarianDataset(Dataset):
    """ Diabetes dataset."""
    # Initialize your data, download, etc.
    def __init__(self):
        #Citim setul de date
        df=pd.read_csv("caesarian.csv",header=None, dtype=np.float32)
        data = normalize(df.values, axis=0, norm='max')
        xy = torch.from_numpy(data)
        self.len = xy.shape[0]
        #Vom folosi ca input toate valorile mai puțin ultima coloană
        self.x_data = xy[:, 0:-1]
        #normalizam prima coloana
        self.x_data = self.x_data / self.x_data.max(0, keepdim=True)[0]
        #print(self.x_data)
        #Vom folosi ca output ultima coloană
        self.y_data = xy[:, [-1]]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [9]:
dataset = CaesarianDataset()
#DataLoader - un utilitar ce ne ajută să împărțim setul de date pe batch-uri și astfel să facem antrenare în mod Mini-Batch
train_loader = DataLoader(dataset=dataset,
                          batch_size=16,
                          shuffle=True,
                          num_workers=1)

In [10]:
dataset[0]

(tensor([0.5500, 0.2500, 0.0000, 1.0000, 0.0000]), tensor([0.]))

In [11]:
class Model(nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l1 = nn.Linear(5, 6)
        self.l2 = nn.Linear(6, 4)
        self.l3 = nn.Linear(4, 1)
        torch.manual_seed(1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        
        
        out2 = nn.functional.relu(input=self.l2(out1), inplace=False)
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

In [12]:
model=Model()

In [13]:
criterion = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.05)

In [14]:
# Training loop
for epoch in range(1000):
  loss_arr = []
  for i, data in enumerate(train_loader, 0):
      # get the inputs
      inputs, labels = data

      # Forward pass: Compute predicted y by passing x to the model
      y_pred = model(inputs)
      
      # Compute and print loss
      loss = criterion(y_pred, labels)
      print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')
      loss_arr.append(loss)
      # Zero gradients, perform a backward pass, and update the weights.
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  mean = sum(loss_arr)/len(loss_arr) 
  print(f'Mean of loss / batch {mean}') 

Epoch 1 | Batch: 1 | Loss: 10.7739
Epoch 1 | Batch: 2 | Loss: 12.5602
Epoch 1 | Batch: 3 | Loss: 11.2804
Epoch 1 | Batch: 4 | Loss: 10.5310
Epoch 1 | Batch: 5 | Loss: 11.3492
Mean of loss / batch 11.29896068572998
Epoch 2 | Batch: 1 | Loss: 10.9681
Epoch 2 | Batch: 2 | Loss: 11.5270
Epoch 2 | Batch: 3 | Loss: 10.8147
Epoch 2 | Batch: 4 | Loss: 10.7241
Epoch 2 | Batch: 5 | Loss: 10.6748
Mean of loss / batch 10.941723823547363
Epoch 3 | Batch: 1 | Loss: 11.3194
Epoch 3 | Batch: 2 | Loss: 10.9658
Epoch 3 | Batch: 3 | Loss: 11.4979
Epoch 3 | Batch: 4 | Loss: 10.6639
Epoch 3 | Batch: 5 | Loss: 10.3980
Mean of loss / batch 10.969015121459961
Epoch 4 | Batch: 1 | Loss: 11.3780
Epoch 4 | Batch: 2 | Loss: 10.9693
Epoch 4 | Batch: 3 | Loss: 10.0857
Epoch 4 | Batch: 4 | Loss: 11.0277
Epoch 4 | Batch: 5 | Loss: 11.4175
Mean of loss / batch 10.975618362426758
Epoch 5 | Batch: 1 | Loss: 10.6516
Epoch 5 | Batch: 2 | Loss: 10.9851
Epoch 5 | Batch: 3 | Loss: 11.3141
Epoch 5 | Batch: 4 | Loss: 10.6943
E

Epoch 38 | Batch: 4 | Loss: 10.2680
Epoch 38 | Batch: 5 | Loss: 11.0007
Mean of loss / batch 10.974699020385742
Epoch 39 | Batch: 1 | Loss: 11.7081
Epoch 39 | Batch: 2 | Loss: 10.2491
Epoch 39 | Batch: 3 | Loss: 11.3945
Epoch 39 | Batch: 4 | Loss: 10.3448
Epoch 39 | Batch: 5 | Loss: 11.4109
Mean of loss / batch 11.021490097045898
Epoch 40 | Batch: 1 | Loss: 12.5870
Epoch 40 | Batch: 2 | Loss: 11.0387
Epoch 40 | Batch: 3 | Loss: 10.9155
Epoch 40 | Batch: 4 | Loss: 10.4435
Epoch 40 | Batch: 5 | Loss: 10.2971
Mean of loss / batch 11.05634880065918
Epoch 41 | Batch: 1 | Loss: 11.0234
Epoch 41 | Batch: 2 | Loss: 10.2216
Epoch 41 | Batch: 3 | Loss: 10.1257
Epoch 41 | Batch: 4 | Loss: 10.0628
Epoch 41 | Batch: 5 | Loss: 14.0132
Mean of loss / batch 11.08936595916748
Epoch 42 | Batch: 1 | Loss: 10.7059
Epoch 42 | Batch: 2 | Loss: 10.0483
Epoch 42 | Batch: 3 | Loss: 10.5928
Epoch 42 | Batch: 4 | Loss: 11.9663
Epoch 42 | Batch: 5 | Loss: 11.6110
Mean of loss / batch 10.984879493713379
Epoch 43 |

Epoch 76 | Batch: 1 | Loss: 11.2701
Epoch 76 | Batch: 2 | Loss: 10.2432
Epoch 76 | Batch: 3 | Loss: 11.0042
Epoch 76 | Batch: 4 | Loss: 10.9904
Epoch 76 | Batch: 5 | Loss: 11.3245
Mean of loss / batch 10.966461181640625
Epoch 77 | Batch: 1 | Loss: 11.2412
Epoch 77 | Batch: 2 | Loss: 10.7465
Epoch 77 | Batch: 3 | Loss: 10.9663
Epoch 77 | Batch: 4 | Loss: 10.6943
Epoch 77 | Batch: 5 | Loss: 10.9736
Mean of loss / batch 10.924383163452148
Epoch 78 | Batch: 1 | Loss: 11.2752
Epoch 78 | Batch: 2 | Loss: 10.7210
Epoch 78 | Batch: 3 | Loss: 10.6733
Epoch 78 | Batch: 4 | Loss: 10.9794
Epoch 78 | Batch: 5 | Loss: 10.9743
Mean of loss / batch 10.924641609191895
Epoch 79 | Batch: 1 | Loss: 10.9710
Epoch 79 | Batch: 2 | Loss: 12.1521
Epoch 79 | Batch: 3 | Loss: 10.6702
Epoch 79 | Batch: 4 | Loss: 11.2938
Epoch 79 | Batch: 5 | Loss: 10.4525
Mean of loss / batch 11.107943534851074
Epoch 80 | Batch: 1 | Loss: 10.9862
Epoch 80 | Batch: 2 | Loss: 11.9837
Epoch 80 | Batch: 3 | Loss: 10.8099
Epoch 80 | B

Epoch 113 | Batch: 3 | Loss: 11.0006
Epoch 113 | Batch: 4 | Loss: 10.8440
Epoch 113 | Batch: 5 | Loss: 10.3222
Mean of loss / batch 10.982478141784668
Epoch 114 | Batch: 1 | Loss: 10.6214
Epoch 114 | Batch: 2 | Loss: 11.4062
Epoch 114 | Batch: 3 | Loss: 11.2943
Epoch 114 | Batch: 4 | Loss: 10.1955
Epoch 114 | Batch: 5 | Loss: 11.4170
Mean of loss / batch 10.986859321594238
Epoch 115 | Batch: 1 | Loss: 10.3248
Epoch 115 | Batch: 2 | Loss: 11.0148
Epoch 115 | Batch: 3 | Loss: 11.3773
Epoch 115 | Batch: 4 | Loss: 10.9707
Epoch 115 | Batch: 5 | Loss: 10.9687
Mean of loss / batch 10.931254386901855
Epoch 116 | Batch: 1 | Loss: 10.9674
Epoch 116 | Batch: 2 | Loss: 11.2456
Epoch 116 | Batch: 3 | Loss: 10.7429
Epoch 116 | Batch: 4 | Loss: 10.4084
Epoch 116 | Batch: 5 | Loss: 11.3680
Mean of loss / batch 10.94644546508789
Epoch 117 | Batch: 1 | Loss: 10.6699
Epoch 117 | Batch: 2 | Loss: 10.9805
Epoch 117 | Batch: 3 | Loss: 10.6522
Epoch 117 | Batch: 4 | Loss: 10.9877
Epoch 117 | Batch: 5 | Loss

Epoch 150 | Batch: 1 | Loss: 11.1984
Epoch 150 | Batch: 2 | Loss: 10.9734
Epoch 150 | Batch: 3 | Loss: 10.9704
Epoch 150 | Batch: 4 | Loss: 10.7590
Epoch 150 | Batch: 5 | Loss: 10.6978
Mean of loss / batch 10.919794082641602
Epoch 151 | Batch: 1 | Loss: 10.6584
Epoch 151 | Batch: 2 | Loss: 11.3370
Epoch 151 | Batch: 3 | Loss: 12.0966
Epoch 151 | Batch: 4 | Loss: 10.7205
Epoch 151 | Batch: 5 | Loss: 10.6610
Mean of loss / batch 11.094693183898926
Epoch 152 | Batch: 1 | Loss: 11.3332
Epoch 152 | Batch: 2 | Loss: 10.1262
Epoch 152 | Batch: 3 | Loss: 10.5991
Epoch 152 | Batch: 4 | Loss: 11.0351
Epoch 152 | Batch: 5 | Loss: 11.8188
Mean of loss / batch 10.982465744018555
Epoch 153 | Batch: 1 | Loss: 10.4174
Epoch 153 | Batch: 2 | Loss: 10.6227
Epoch 153 | Batch: 3 | Loss: 10.6097
Epoch 153 | Batch: 4 | Loss: 11.4379
Epoch 153 | Batch: 5 | Loss: 11.6509
Mean of loss / batch 10.947700500488281
Epoch 154 | Batch: 1 | Loss: 10.9670
Epoch 154 | Batch: 2 | Loss: 11.4176
Epoch 154 | Batch: 3 | Los

...

Epoch 850 | Batch: 5 | Loss: 10.5559
Mean of loss / batch 10.993855476379395
Epoch 851 | Batch: 1 | Loss: 10.3424
Epoch 851 | Batch: 2 | Loss: 10.6071
Epoch 851 | Batch: 3 | Loss: 11.0228
Epoch 851 | Batch: 4 | Loss: 11.7824
Epoch 851 | Batch: 5 | Loss: 10.9653
Mean of loss / batch 10.943978309631348
Epoch 852 | Batch: 1 | Loss: 11.4864
Epoch 852 | Batch: 2 | Loss: 10.8232
Epoch 852 | Batch: 3 | Loss: 10.7391
Epoch 852 | Batch: 4 | Loss: 10.6850
Epoch 852 | Batch: 5 | Loss: 10.9759
Mean of loss / batch 10.941919326782227
Epoch 853 | Batch: 1 | Loss: 10.6608
Epoch 853 | Batch: 2 | Loss: 12.0324
Epoch 853 | Batch: 3 | Loss: 11.1553
Epoch 853 | Batch: 4 | Loss: 10.6990
Epoch 853 | Batch: 5 | Loss: 10.4344
Mean of loss / batch 10.996377944946289
Epoch 854 | Batch: 1 | Loss: 10.9893
Epoch 854 | Batch: 2 | Loss: 10.2988
Epoch 854 | Batch: 3 | Loss: 10.1760
Epoch 854 | Batch: 4 | Loss: 11.5658
Epoch 854 | Batch: 5 | Loss: 11.7926
Mean of loss / batch 10.964517593383789
Epoch 855 | Batch: 1 | 

Epoch 888 | Batch: 1 | Loss: 10.4449
Epoch 888 | Batch: 2 | Loss: 10.2707
Epoch 888 | Batch: 3 | Loss: 10.1577
Epoch 888 | Batch: 4 | Loss: 12.5906
Epoch 888 | Batch: 5 | Loss: 11.5766
Mean of loss / batch 11.008084297180176
Epoch 889 | Batch: 1 | Loss: 11.1647
Epoch 889 | Batch: 2 | Loss: 10.9835
Epoch 889 | Batch: 3 | Loss: 10.4560
Epoch 889 | Batch: 4 | Loss: 10.9801
Epoch 889 | Batch: 5 | Loss: 11.2966
Mean of loss / batch 10.9761962890625
Epoch 890 | Batch: 1 | Loss: 11.4812
Epoch 890 | Batch: 2 | Loss: 10.8260
Epoch 890 | Batch: 3 | Loss: 10.7409
Epoch 890 | Batch: 4 | Loss: 10.9667
Epoch 890 | Batch: 5 | Loss: 10.6913
Mean of loss / batch 10.941225051879883
Epoch 891 | Batch: 1 | Loss: 10.9743
Epoch 891 | Batch: 2 | Loss: 10.6644
Epoch 891 | Batch: 3 | Loss: 11.6742
Epoch 891 | Batch: 4 | Loss: 10.9662
Epoch 891 | Batch: 5 | Loss: 10.5016
Mean of loss / batch 10.956133842468262
Epoch 892 | Batch: 1 | Loss: 10.9790
Epoch 892 | Batch: 2 | Loss: 10.9741
Epoch 892 | Batch: 3 | Loss:

Epoch 924 | Batch: 4 | Loss: 10.9807
Epoch 924 | Batch: 5 | Loss: 10.2553
Mean of loss / batch 10.983091354370117
Epoch 925 | Batch: 1 | Loss: 10.6108
Epoch 925 | Batch: 2 | Loss: 11.4344
Epoch 925 | Batch: 3 | Loss: 11.3127
Epoch 925 | Batch: 4 | Loss: 10.9656
Epoch 925 | Batch: 5 | Loss: 10.4360
Mean of loss / batch 10.951894760131836
Epoch 926 | Batch: 1 | Loss: 10.2649
Epoch 926 | Batch: 2 | Loss: 10.1539
Epoch 926 | Batch: 3 | Loss: 12.6001
Epoch 926 | Batch: 4 | Loss: 10.3606
Epoch 926 | Batch: 5 | Loss: 11.7942
Mean of loss / batch 11.0347318649292
Epoch 927 | Batch: 1 | Loss: 10.4322
Epoch 927 | Batch: 2 | Loss: 10.2625
Epoch 927 | Batch: 3 | Loss: 10.1523
Epoch 927 | Batch: 4 | Loss: 13.1089
Epoch 927 | Batch: 5 | Loss: 11.2210
Mean of loss / batch 11.035379409790039
Epoch 928 | Batch: 1 | Loss: 10.9693
Epoch 928 | Batch: 2 | Loss: 10.5395
Epoch 928 | Batch: 3 | Loss: 11.9391
Epoch 928 | Batch: 4 | Loss: 11.1401
Epoch 928 | Batch: 5 | Loss: 10.6172
Mean of loss / batch 11.0410

Epoch 961 | Batch: 1 | Loss: 9.3931
Epoch 961 | Batch: 2 | Loss: 12.2484
Epoch 961 | Batch: 3 | Loss: 9.8194
Epoch 961 | Batch: 4 | Loss: 12.1371
Epoch 961 | Batch: 5 | Loss: 12.0892
Mean of loss / batch 11.137452125549316
Epoch 962 | Batch: 1 | Loss: 10.9716
Epoch 962 | Batch: 2 | Loss: 11.5845
Epoch 962 | Batch: 3 | Loss: 10.9060
Epoch 962 | Batch: 4 | Loss: 11.1713
Epoch 962 | Batch: 5 | Loss: 10.4974
Mean of loss / batch 11.026143074035645
Epoch 963 | Batch: 1 | Loss: 11.3059
Epoch 963 | Batch: 2 | Loss: 10.9653
Epoch 963 | Batch: 3 | Loss: 10.1811
Epoch 963 | Batch: 4 | Loss: 11.8326
Epoch 963 | Batch: 5 | Loss: 10.6879
Mean of loss / batch 10.994576454162598
Epoch 964 | Batch: 1 | Loss: 10.3287
Epoch 964 | Batch: 2 | Loss: 10.6045
Epoch 964 | Batch: 3 | Loss: 11.4547
Epoch 964 | Batch: 4 | Loss: 10.2938
Epoch 964 | Batch: 5 | Loss: 12.3023
Mean of loss / batch 10.996798515319824
Epoch 965 | Batch: 1 | Loss: 10.7232
Epoch 965 | Batch: 2 | Loss: 11.2624
Epoch 965 | Batch: 3 | Loss:

Mean of loss / batch 10.962138175964355
Epoch 998 | Batch: 1 | Loss: 10.9677
Epoch 998 | Batch: 2 | Loss: 11.2476
Epoch 998 | Batch: 3 | Loss: 11.1915
Epoch 998 | Batch: 4 | Loss: 10.9750
Epoch 998 | Batch: 5 | Loss: 10.3886
Mean of loss / batch 10.954065322875977
Epoch 999 | Batch: 1 | Loss: 10.9863
Epoch 999 | Batch: 2 | Loss: 11.9844
Epoch 999 | Batch: 3 | Loss: 11.1473
Epoch 999 | Batch: 4 | Loss: 10.5864
Epoch 999 | Batch: 5 | Loss: 10.3551
Mean of loss / batch 11.011906623840332
Epoch 1000 | Batch: 1 | Loss: 11.0065
Epoch 1000 | Batch: 2 | Loss: 11.3606
Epoch 1000 | Batch: 3 | Loss: 10.9689
Epoch 1000 | Batch: 4 | Loss: 10.6805
Epoch 1000 | Batch: 5 | Loss: 10.6472
Mean of loss / batch 10.932740211486816
